In [1]:
import os
import sys

os.getcwd()
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(f'{parent_dir}/dsnp_transform')
import hraResultTransform as hrt


In [2]:
import pandas as pd
path = r"C:\Users\A845514\Projects_code\data\hra_result\pull_db"
wide_df = pd.read_pickle(f'{path}/wide_hra_section_chronic_conditions_template4.pkl')



In [3]:
wide_df.columns

Index(['asm_id_value_desc',
       'Are you taking medications or being treated for this condition?',
       'Have you ever been diagnosed with any of the following conditions? (check all that apply)',
       'How long have you had this condition?',
       'If 'other' describe condition 1:', 'If 'other' describe condition 2:',
       'If 'other' describe condition 3:', 'If 'other' describe condition 4:',
       'If 'other' describe condition 5:',
       'If there are other conditions, please describe all "other" conditions',
       'asad_updated_on_dts'],
      dtype='object')

In [7]:
wide_df

,asm_id_value_desc,Are you taking medications or being treated for this condition?,Have you ever been diagnosed with any of the following conditions? (check all that apply),How long have you had this condition?,If 'other' describe condition 1:,If 'other' describe condition 2:,If 'other' describe condition 3:,If 'other' describe condition 4:,If 'other' describe condition 5:,"If there are other conditions, please describe all ""other"" conditions",asad_updated_on_dts
0,,Yes,Bipolar Disorder,5+ years,seizure - had one last year due to alcohol wit...,Allergies,Arthritis,Cramping in both feet.,NaN,"neuropathy, tumor in left cortex of brain (goi...",2024-04-19 13:51:21.306666
1,101039985900,Yes,Chronic Obstructive Pulmonary Disease (COPD)|H...,5+ years,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-06 21:23:42.810121
2,101039986100,Yes,High Blood Pressure,5+ years,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-18 15:11:52.402574
3,101040086100,NaN,Decline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-06 06:19:21.577945
4,101040408900,Yes,High Blood Pressure,3-5 years,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-04 22:17:41.041921
...,...,...,...,...,...,...,...,...,...,...,...
269789,WVAQMZM00001858813,Yes,Anxiety/Panic|Any cancer (besides skin cancer)...,5+ years,Aortic Stenosis,NaN,NaN,NaN,NaN,NaN,2024-01-14 06:51:03.980615
269790,WVAQMZM00001859496,Unsure,Chronic Obstructive Pulmonary Disease (COPD)|E...,5+ years,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-19 00:29:02.833053
269791,WVAQMZM00001861311,Yes,Anxiety/Panic|Congestive Heart Failure (CHF)|D...,5+ years,Panic attacks,NaN,NaN,NaN,NaN,NaN,2023-11-16 18:59:53.191989
269792,WVAQMZM00002121598,No,Any cancer (besides skin cancer)|Asthma|Diabetes|,Less than a year,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-12 19:51:20.214399


In [8]:
wide_df.columns

Index(['asm_id_value_desc',
       'Are you taking medications or being treated for this condition?',
       'Have you ever been diagnosed with any of the following conditions? (check all that apply)',
       'How long have you had this condition?',
       'If 'other' describe condition 1:', 'If 'other' describe condition 2:',
       'If 'other' describe condition 3:', 'If 'other' describe condition 4:',
       'If 'other' describe condition 5:',
       'If there are other conditions, please describe all "other" conditions',
       'asad_updated_on_dts'],
      dtype='object')

In [9]:
pd.set_option("display.max_columns", None)

In [34]:
def clean_chronic_condition(df):
    '''clean template 4, chronic_condition section, use it after clean_hra_qa'''

    question_map = {'asm_id_value_desc':"dsnp_member_id",
    "Have you ever been diagnosed with any of the following conditions? (check all that apply)": "diagnosed_conditions",
    "If other describe condition 1:": 'other_diagnosed_1',
    "If other describe condition 2:": 'other_diagnosed_2',
    "If other describe condition 3:": 'other_diagnosed_3',
    "If other describe condition 4:": 'other_diagnosed_4',
    "If other describe condition 5:": 'other_diagnosed_5',
    "If there are other conditions, please describe all other conditions": 'other_diagnosed_6'}

    df.columns = df.columns.str.strip().str.replace("'", "").str.replace('"', "")
    cleaned_df = df.rename(columns = question_map)
    print(cleaned_df.columns)
    cleaned_df = cleaned_df.apply(lambda x: x.str.lower().str.strip() if x.dtype =='object' and x.name !='dsnp_member_id' else x)
    concatenated_col = ['other_diagnosed_1','other_diagnosed_2','other_diagnosed_3','other_diagnosed_4','other_diagnosed_5','other_diagnosed_6']
    cleaned_df["other_diagnosed"] = cleaned_df[concatenated_col].apply(lambda x: '|'.join(x.dropna().astype(str)), axis=1)
    cleaned_df["hra_section"] = "chronic_condition"
    cleaned_df = cleaned_df[["dsnp_member_id", "diagnosed_conditions", "other_diagnosed", "asad_updated_on_dts", "hra_section"]]
    return cleaned_df

In [35]:
cleaned_df = clean_chronic_condition(wide_df)

Index(['dsnp_member_id',
       'Are you taking medications or being treated for this condition?',
       'diagnosed_conditions', 'How long have you had this condition?',
       'other_diagnosed_1', 'other_diagnosed_2', 'other_diagnosed_3',
       'other_diagnosed_4', 'other_diagnosed_5', 'other_diagnosed_6',
       'asad_updated_on_dts'],
      dtype='object')


In [36]:
cleaned_df

,dsnp_member_id,diagnosed_conditions,other_diagnosed,asad_updated_on_dts,hra_section
0,,bipolar disorder,seizure - had one last year due to alcohol wit...,2024-04-19 13:51:21.306666,chronic_condition
1,101039985900,chronic obstructive pulmonary disease (copd)|h...,,2024-02-06 21:23:42.810121,chronic_condition
2,101039986100,high blood pressure,,2024-03-18 15:11:52.402574,chronic_condition
3,101040086100,decline,,2024-01-06 06:19:21.577945,chronic_condition
4,101040408900,high blood pressure,,2024-03-04 22:17:41.041921,chronic_condition
...,...,...,...,...,...
269789,WVAQMZM00001858813,anxiety/panic|any cancer (besides skin cancer)...,aortic stenosis,2024-01-14 06:51:03.980615,chronic_condition
269790,WVAQMZM00001859496,chronic obstructive pulmonary disease (copd)|e...,,2023-12-19 00:29:02.833053,chronic_condition
269791,WVAQMZM00001861311,anxiety/panic|congestive heart failure (chf)|d...,panic attacks,2023-11-16 18:59:53.191989,chronic_condition
269792,WVAQMZM00002121598,any cancer (besides skin cancer)|asthma|diabetes|,,2023-12-12 19:51:20.214399,chronic_condition


In [32]:
cleaned_df.iloc[0]['other_diagnosed']

'seizure - had one last year due to alcohol withdrawal. states he is not drinking currently|allergies|arthritis|cramping in both feet.|neuropathy, tumor in left cortex of brain (going blind and deaf) - leukocytes'